In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = self.data.iloc[idx, :-1].values.astype(float)
        label = self.data.iloc[idx, -1].astype(float)
        return {'features': torch.tensor(features), 'label': torch.tensor(label)}

csv_file = 'handMCsmall-test.csv'  
train_set = CustomDataset('handMCsmall-train.csv')
test_set  = CustomDataset('handMCsmall-test.csv')
#print(dataset[0])

# Split dataset into train and test sets
#train_set, test_set = train_test_split(dataset, test_size=0.2, shuffle=True)
#print(train_set[0:5])



In [4]:
# Define the neural network
class FunctionApproximator(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FunctionApproximator, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
class DeepFunctionApproximator(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(DeepFunctionApproximator, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Define model, loss function, and optimizer
input_dim = len(train_set[0]['features'])
output_dim = 1  # Assuming a single output
hidden_dim = 32 # Adjust according to your problem
#model = FunctionApproximator(input_dim, hidden_dim, output_dim)
model = DeepFunctionApproximator(input_dim, 64, 16, output_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100
batch_size = 4096
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for batch in train_loader:
        features, labels = batch['features'].to(device), batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(features.float())
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
model.to(cpu)
# Evaluate the model
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
test_loss = 0
with torch.no_grad():
    for batch in test_loader:
        features, labels = batch['features'], batch['label']
        outputs = model(features.float())
        test_loss += criterion(outputs.squeeze(), labels.float()).item()

test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.4f}')

cuda
Epoch [1/100], Loss: 0.0769
Epoch [2/100], Loss: 0.0693
Epoch [3/100], Loss: 0.0612
Epoch [4/100], Loss: 0.0569
Epoch [5/100], Loss: 0.0576
Epoch [6/100], Loss: 0.0547
Epoch [7/100], Loss: 0.0524
Epoch [8/100], Loss: 0.0511
Epoch [9/100], Loss: 0.0443
Epoch [10/100], Loss: 0.0462
Epoch [11/100], Loss: 0.0450
Epoch [12/100], Loss: 0.0413
Epoch [13/100], Loss: 0.0421
Epoch [14/100], Loss: 0.0391
Epoch [15/100], Loss: 0.0390
Epoch [16/100], Loss: 0.0373
Epoch [17/100], Loss: 0.0345
Epoch [18/100], Loss: 0.0365
Epoch [19/100], Loss: 0.0367
Epoch [20/100], Loss: 0.0385
Epoch [21/100], Loss: 0.0351
Epoch [22/100], Loss: 0.0431
Epoch [23/100], Loss: 0.0447


KeyboardInterrupt: 

In [32]:
manualinput = torch.tensor([[2,3,6,2,11,4,11,1,4,3]]).float()   # answer is 0.044
greathand = torch.tensor([[14,1,14,2,13,4,12,2,11,5]]).float()  # a really strong hand
decenthand = torch.tensor([[12,1,13,3,9,2,5,2,11,2]]).float()   # a 50/50 hand: answer is 0.457

model.eval()
with torch.no_grad():
    output = model(manualinput)
    print(output)
    output = model(greathand)
    print(output)
    output = model(decenthand)
    print(output)

tensor([[0.0484]])
tensor([[0.9678]])
tensor([[0.5003]])
